In [1]:
import os
import pandas as pd
from model_config import Path
import plotly.graph_objects as go 
import numpy as np
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

- All Fiches

In [2]:
model_num = 1
data_num = 1

In [3]:
# Define numerical features to scale
numerical_features = [
                    "Close",
                    'Open', 'High', 'Low', 'Average',  'Change', 'Volume', 'Volume Change',
                    'EMA', 'SMA',  'MACD', 'BB_upper', 'BB_middle', 'BB_lower'
                      ]

In [4]:
scaler = StandardScaler()

In [5]:
# Specify the folder path containing your CSV files
folder_path = os.path.join(os.getcwd(), "data", "raw")

# Get a list of all CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Initialize an empty list to hold individual DataFrames
dataframes = []

In [6]:
# Loop through each CSV file in the folder
for csv_file in csv_files:
    # Construct full file path
    file_path = os.path.join(folder_path, csv_file)
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    df = df.iloc[1:].reset_index(drop=True)
    
    # Extract the Asset_ID from the file name (assuming file name is the Asset_ID)
    asset_id = os.path.splitext(csv_file)[0].split("_")[0]  # Removes the '.csv' extension
    
    # Add the Asset_ID column to the DataFrame
    df['Asset_ID'] = asset_id
    
    # Append the DataFrame to the list
    dataframes.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the combined DataFrame
combined_df.head()

,Date,Open,High,Low,Average,Close,Change,Volume,Volume Change,Asset_ID
0,2024-04-13 01:00,0.000049,0.000051,0.000048,0.000050,0.000049,-0.173826,165230.754141,-35.042142,ANDY
1,2024-04-13 02:00,0.000049,0.000050,0.000044,0.000047,0.000045,-7.308112,174371.859513,5.532327,ANDY
2,2024-04-13 03:00,0.000045,0.000046,0.000044,0.000045,0.000046,3.174317,35711.614073,-79.519852,ANDY
3,2024-04-13 04:00,0.000046,0.000050,0.000046,0.000048,0.000047,0.842223,78398.220456,119.531440,ANDY
4,2024-04-13 05:00,0.000047,0.000060,0.000045,0.000052,0.000047,-0.538570,297045.536758,278.893213,ANDY


In [7]:
# Обработка пропущенных значений
combined_df.fillna(method='ffill', inplace=True)
combined_df.fillna(method='bfill', inplace=True)

C:\Users\McHomak\AppData\Local\Temp\ipykernel_4848\3639975876.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df.fillna(method='ffill', inplace=True)
C:\Users\McHomak\AppData\Local\Temp\ipykernel_4848\3639975876.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df.fillna(method='bfill', inplace=True)


In [8]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103954 entries, 0 to 103953
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           103954 non-null  object 
 1   Open           103954 non-null  float64
 2   High           103954 non-null  float64
 3   Low            103954 non-null  float64
 4   Average        103954 non-null  float64
 5   Close          103954 non-null  float64
 6   Change         103954 non-null  float64
 7   Volume         103954 non-null  float64
 8   Volume Change  103954 non-null  float64
 9   Asset_ID       103954 non-null  object 
dtypes: float64(8), object(2)
memory usage: 7.9+ MB


In [9]:
# Convert 'Date' column to datetime if not already
combined_df['Date'] = pd.to_datetime(combined_df['Date'])

# Sort by Asset_ID and Date
combined_df.sort_values(by=['Asset_ID', 'Date'], inplace=True)

# Reset index after sorting
combined_df.reset_index(drop=True, inplace=True)

In [10]:
combined_df = combined_df.groupby('Asset_ID', group_keys=False)

In [11]:
import pandas_ta as ta

# Function to calculate indicators for each asset
def add_technical_indicators(group):
    group['EMA'] = ta.ema(group['Close'], length=14)
    group['SMA'] = ta.sma(group['Close'], length=14)
    group['RSI'] = ta.rsi(group['Close'], length=14)
    group['MACD'] = ta.macd(group['Close'])['MACD_12_26_9']
    bbands = ta.bbands(group['Close'], length=20)
    group['BB_upper'] = bbands['BBU_20_2.0']
    group['BB_middle'] = bbands['BBM_20_2.0']
    group['BB_lower'] = bbands['BBL_20_2.0']
    return group

# Apply the function to each group (asset)
combined_df = combined_df.apply(add_technical_indicators)
combined_df.fillna(method='bfill', inplace=True)

C:\Users\McHomak\AppData\Local\Temp\ipykernel_4848\1161799361.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  combined_df = combined_df.apply(add_technical_indicators)
C:\Users\McHomak\AppData\Local\Temp\ipykernel_4848\1161799361.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df.fillna(method='bfill', inplace=True)


In [12]:
# Extract time components
combined_df['Hour'] = combined_df['Date'].dt.hour
combined_df['Day'] = combined_df['Date'].dt.dayofweek  # 0 = Monday
combined_df['Month'] = combined_df['Date'].dt.month

# Cyclical encoding
combined_df['Hour_sin'] = np.sin(2 * np.pi * combined_df['Hour']/24)
combined_df['Hour_cos'] = np.cos(2 * np.pi * combined_df['Hour']/24)

combined_df['Day_sin'] = np.sin(2 * np.pi * combined_df['Day']/7)
combined_df['Day_cos'] = np.cos(2 * np.pi * combined_df['Day']/7)

combined_df['Month_sin'] = np.sin(2 * np.pi * combined_df['Month']/12)
combined_df['Month_cos'] = np.cos(2 * np.pi * combined_df['Month']/12)

In [13]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
combined_df['Asset_ID_encoded'] = label_encoder.fit_transform(combined_df['Asset_ID'])

In [14]:
combined_df["Close_orig"] = combined_df["Close"]

In [15]:
def scale_data(df, scaler):
    if scaler in ["log"]:
        def scale_group(group):
            # Применяем логарифмическую трансформацию к числовым признакам
            group[numerical_features] = np.log(group[numerical_features] + 1e-6)  # Добавляем небольшое значение для избежания логарифма от 0
            return group
    else:
        def scale_group(group):
            group[numerical_features] = scaler.fit_transform(group[numerical_features])
            return group  

    df = df.groupby('Asset_ID', group_keys=False).apply(scale_group).reset_index(drop=True)
    return df

In [16]:
combined_df = scale_data(df = combined_df, scaler= scaler)

C:\Users\McHomak\AppData\Local\Temp\ipykernel_4848\2103416537.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Asset_ID', group_keys=False).apply(scale_group).reset_index(drop=True)


In [17]:
# Функция для замены нулей на среднее
def replace_zeros_with_mean(df, columns):
    for column in columns:
        df[column].replace(0, np.nan, inplace=True)
        df[column].interpolate(method='linear', inplace=True)
        df[column].fillna(method='bfill', inplace=True)
        df[column].fillna(method='ffill', inplace=True)
    return df

# Задаем столбцы, в которых нужно заменить нули
columns_to_replace = [ 
                      "Close_orig",   
                      "Close", 'Open', 'High', 'Low', 'Average',  'Change', 'Volume', 'Volume Change',
                      'EMA', 'SMA', 'RSI', 'MACD', 'BB_upper', 'BB_middle', 'BB_lower'
                      ]

In [18]:
# Замена нулевых значений на средние с интерполяцией для каждого Asset_ID
combined_df = combined_df.groupby('Asset_ID', group_keys=False).apply(lambda group: replace_zeros_with_mean(group, columns_to_replace)).reset_index(drop=True)

C:\Users\McHomak\AppData\Local\Temp\ipykernel_4848\4021808225.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].replace(0, np.nan, inplace=True)
C:\Users\McHomak\AppData\Local\Temp\ipykernel_4848\4021808225.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

In [19]:
zero_close_prices = combined_df[combined_df['Close'] == 0]
print(f"Number of zero 'Close' prices after scaling: {len(zero_close_prices)}")

Number of zero 'Close' prices after scaling: 0


In [20]:
combined_df

,Date,Open,High,Low,Average,Close,Change,Volume,Volume Change,Asset_ID,...,Day,Month,Hour_sin,Hour_cos,Day_sin,Day_cos,Month_sin,Month_cos,Asset_ID_encoded,Close_orig
0,2024-04-13 01:00:00,-1.230904,-1.197139,-1.227398,-1.213365,-1.233058,-0.075295,0.223341,-0.211544,ANDY,...,5,4,0.258819,9.659258e-01,-0.974928,-0.222521,0.866025,-0.5,0,0.000049
1,2024-04-13 02:00:00,-1.232477,-1.216596,-1.308983,-1.262980,-1.298704,-2.145742,0.269265,-0.150410,ANDY,...,5,4,0.500000,8.660254e-01,-0.974928,-0.222521,0.866025,-0.5,0,0.000045
2,2024-04-13 03:00:00,-1.298115,-1.280958,-1.298183,-1.290982,-1.272283,0.896372,-0.427343,-0.278558,ANDY,...,5,4,0.707107,7.071068e-01,-0.974928,-0.222521,0.866025,-0.5,0,0.000046
3,2024-04-13 04:00:00,-1.271697,-1.222140,-1.267419,-1.245690,-1.265048,0.219573,-0.212892,0.021352,ANDY,...,5,4,0.866025,5.000000e-01,-0.974928,-0.222521,0.866025,-0.5,0,0.000047
4,2024-04-13 05:00:00,-1.264463,-1.034832,-1.287573,-1.158726,-1.269711,-0.181148,0.885559,0.261462,ANDY,...,5,4,0.965926,2.588190e-01,-0.974928,-0.222521,0.866025,-0.5,0,0.000047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103949,2024-10-12 16:00:00,2.972005,2.928544,2.998724,2.964046,2.994750,0.283391,-0.339070,-0.102031,WTAO,...,5,10,-0.866025,-5.000000e-01,-0.974928,-0.222521,-0.866025,0.5,25,671.613074
103950,2024-10-12 17:00:00,2.996957,3.000082,2.848789,2.926220,2.966157,-0.345206,2.891522,0.093538,WTAO,...,5,10,-0.965926,-2.588190e-01,-0.974928,-0.222521,-0.866025,0.5,25,668.685056
103951,2024-10-12 18:00:00,2.968346,2.918469,3.006707,2.962887,2.938882,-0.331173,-0.403076,-0.103266,WTAO,...,5,10,-1.000000,-1.836970e-16,-0.974928,-0.222521,-0.866025,0.5,25,665.891859
103952,2024-10-12 19:00:00,2.941053,2.954062,3.006707,2.980911,2.943979,0.050237,0.087180,-0.052370,WTAO,...,5,10,-0.965926,2.588190e-01,-0.974928,-0.222521,-0.866025,0.5,25,666.413883


In [21]:
combined_df.sort_values(by=['Asset_ID', 'Date'], inplace=True)
combined_df.drop(columns= ['Date', "Asset_ID"], axis=1, inplace=True)

In [22]:
combined_df.to_csv(Path["dataset"](model_num, data_num), index= False)

In [23]:
df = pd.read_csv(Path["dataset"](model_num, data_num))

In [24]:
df.columns

Index(['Open', 'High', 'Low', 'Average', 'Close', 'Change', 'Volume',
       'Volume Change', 'EMA', 'SMA', 'RSI', 'MACD', 'BB_upper', 'BB_middle',
       'BB_lower', 'Hour', 'Day', 'Month', 'Hour_sin', 'Hour_cos', 'Day_sin',
       'Day_cos', 'Month_sin', 'Month_cos', 'Asset_ID_encoded', 'Close_orig'],
      dtype='object')

In [25]:
df

,Open,High,Low,Average,Close,Change,Volume,Volume Change,EMA,SMA,...,Day,Month,Hour_sin,Hour_cos,Day_sin,Day_cos,Month_sin,Month_cos,Asset_ID_encoded,Close_orig
0,-1.230904,-1.197139,-1.227398,-1.213365,-1.233058,-0.075295,0.223341,-0.211544,-1.161650,-1.159325,...,5,4,0.258819,9.659258e-01,-0.974928,-0.222521,0.866025,-0.5,0,0.000049
1,-1.232477,-1.216596,-1.308983,-1.262980,-1.298704,-2.145742,0.269265,-0.150410,-1.161650,-1.159325,...,5,4,0.500000,8.660254e-01,-0.974928,-0.222521,0.866025,-0.5,0,0.000045
2,-1.298115,-1.280958,-1.298183,-1.290982,-1.272283,0.896372,-0.427343,-0.278558,-1.161650,-1.159325,...,5,4,0.707107,7.071068e-01,-0.974928,-0.222521,0.866025,-0.5,0,0.000046
3,-1.271697,-1.222140,-1.267419,-1.245690,-1.265048,0.219573,-0.212892,0.021352,-1.161650,-1.159325,...,5,4,0.866025,5.000000e-01,-0.974928,-0.222521,0.866025,-0.5,0,0.000047
4,-1.264463,-1.034832,-1.287573,-1.158726,-1.269711,-0.181148,0.885559,0.261462,-1.161650,-1.159325,...,5,4,0.965926,2.588190e-01,-0.974928,-0.222521,0.866025,-0.5,0,0.000047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103949,2.972005,2.928544,2.998724,2.964046,2.994750,0.283391,-0.339070,-0.102031,2.860254,2.847981,...,5,10,-0.866025,-5.000000e-01,-0.974928,-0.222521,-0.866025,0.5,25,671.613074
103950,2.996957,3.000082,2.848789,2.926220,2.966157,-0.345206,2.891522,0.093538,2.877940,2.870331,...,5,10,-0.965926,-2.588190e-01,-0.974928,-0.222521,-0.866025,0.5,25,668.685056
103951,2.968346,2.918469,3.006707,2.962887,2.938882,-0.331173,-0.403076,-0.103266,2.889600,2.886343,...,5,10,-1.000000,-1.836970e-16,-0.974928,-0.222521,-0.866025,0.5,25,665.891859
103952,2.941053,2.954062,3.006707,2.980911,2.943979,0.050237,0.087180,-0.052370,2.900391,2.900699,...,5,10,-0.965926,2.588190e-01,-0.974928,-0.222521,-0.866025,0.5,25,666.413883
